In [ ]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.image as mpimg
import os
from sklearn.model_selection import train_test_split, GridSearchCV, cross_val_score
from sklearn.preprocessing import scale
from sklearn.neural_network import MLPClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA
from sklearn.metrics import accuracy_score, confusion_matrix
from sklearn.preprocessing import StandardScaler

In [ ]:
seed = 1
np.random.seed(seed)

In [ ]:
scaler = StandardScaler()
df = pd.read_csv("https://github.com/mpourhoma/CS4662/raw/master/CTG_clean.csv")
X = df.iloc[:, :-1]  
y = df.iloc[:, -1]   

In [ ]:
X_scaled = scaler.fit_transform(X)

In [ ]:
ann = MLPClassifier(hidden_layer_sizes=(30,), activation='logistic', solver='adam', 
                    alpha=1, learning_rate_init=0.02, random_state=1, verbose=False)

In [ ]:
cv_scores = cross_val_score(ann, X_scaled, y, cv=10, scoring='accuracy')
ann_accuracy = np.mean(cv_scores)
print(f"ANN Accuracy with 30 neurons: {ann_accuracy:.4f}")

In [ ]:
param_grid = {'hidden_layer_sizes': [(i,) for i in range(5, 255, 5)]}
gs_ann = GridSearchCV(ann, param_grid, cv=10, scoring='accuracy')
gs_ann.fit(X_scaled, y)
print(f"Best ANN Accuracy: {gs_ann.best_score_:.4f} with {gs_ann.best_params_}")

Problem 2: Face Recognition using SVM and PCA

In [ ]:
images = "/Users/markraeven17/COMPSCI/MACHINE LEARNING/Face"

In [ ]:
labels_file = os.path.join(images, "label.csv")  
print(f"Looking for: {labels_file}")
labels_df = pd.read_csv(labels_file)

In [ ]:
X = []
y = labels_df["Label"].values

In [ ]:
for i in range(len(y)):
    img_path = os.path.join(images, f"{i}.jpg")  
    if os.path.exists(img_path):
        img = mpimg.imread(img_path)  
        X.append(img.flatten())  
    else:
        print(f"Warning: Image {img_path} not found!")

In [ ]:
X = np.array(X)
y = np.array(y)

In [ ]:
X_scaled = scale(X)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X_scaled, y, test_size=0.25, random_state=5)

PCA dimensionality reduction

In [ ]:
k = 50
pca = PCA(n_components=k)
X_train_new = pca.fit_transform(X_train)
X_test_new = pca.transform(X_test)

In [ ]:
svm = SVC(C=1, kernel='rbf', gamma=0.0005, random_state=1)
svm.fit(X_train_new, y_train)
y_pred = svm.predict(X_test_new)

In [ ]:
svm_accuracy = accuracy_score(y_test, y_pred)
cm = confusion_matrix(y_test, y_pred)
print(f"SVM Accuracy: {svm_accuracy:.4f}")
print("Confusion Matrix:\n", cm)

In [ ]:
param_grid = {'C': [0.1, 1, 10, 100, 1e3, 5e3, 1e4, 5e4, 1e5]}
gs_svm = GridSearchCV(SVC(kernel='rbf', gamma=0.0005, random_state=1), param_grid, cv=10, scoring='accuracy')
X_pca = pca.fit_transform(X_scaled)
gs_svm.fit(X_pca, y)
print(f"Best SVM Accuracy: {gs_svm.best_score_:.4f} with {gs_svm.best_params_}")